###Match/mismatch predictive coding stimuli

In [1]:
from PIL import Image
from PIL import ImageDraw as Draw
from PIL import ImageFilter
from skimage.filter import gaussian_filter
import numpy as np
from glob import glob
from os import path
from object_parsing.src.image_objects import make_a_blank
import scipy.io as sio

%pylab inline

Populating the interactive namespace from numpy and matplotlib


#### Parameters and code

In [2]:
imgX_fname = '%0.6d.png'
savedir = '/Users/tnaselar/Data/Presentation/predictive.coding.match.nonmatch/four_windows/'
picdir = savedir+'base_images/'

In [3]:
image_scale = 768
image_shape = np.array((1,1))*image_scale ## <<units of image_scale
occluder_locations = {'northwest': (.25*image_scale, .25*image_scale), 
                      'northeast': (.25*image_scale, .75*image_scale),
                      'southwest': (.75*image_scale, .25*image_scale),
                      'southeast': (.75*image_scale, .75*image_scale)}
occluder_radius = .2*image_scale ##units of image_scale
occluder_luminance = 127 ##8-bit scale
isi_luminance = 127
pyplot.set_cmap('gray')

##--run parameters
FADE_IN_FRAMES = 4
FADE_OUT_FRAMES = 3
OCCLUDER_LOCATIONS = len(occluder_locations.keys())
IMAGES_PER_SET = 2 ##assume 1 set per occluder
LOOPS = 3 ##we need this because the reps per set are pegged to the number of images
MATCH_REPS_PER_SET = IMAGES_PER_SET-2
IMAGES_PER_CYCLE = 4  ##(base, imagery_screen, patch, isi)
TRs_PER_IMAGE = 1
TR = 1.5 ##seconds
CYCLES_PER_RUN = (IMAGES_PER_SET**2+IMAGES_PER_SET*MATCH_REPS_PER_SET)*OCCLUDER_LOCATIONS*LOOPS
print 'cycles per run: %d' %(CYCLES_PER_RUN)
CYCLE_DURATION = IMAGES_PER_CYCLE*TRs_PER_IMAGE*TR
print 'cycle duration (sec) : %f' %(CYCLE_DURATION)
RUN_DURATION = CYCLES_PER_RUN*CYCLE_DURATION+(FADE_IN_FRAMES+FADE_OUT_FRAMES)*TR
print 'run duration (min) %f' %(RUN_DURATION / 60.)
ISI_LAM = 0.2

RUNS = 8

print 'total experiment duration (min): %f' %( RUNS*RUN_DURATION / 60. )

UNIQUE_IMAGES = IMAGES_PER_SET*OCCLUDER_LOCATIONS
print '%d images will be imagined %d times each' %(UNIQUE_IMAGES, (MATCH_REPS_PER_SET+1)*LOOPS*RUNS)



cycles per run: 48
cycle duration (sec) : 6.000000
run duration (min) 4.975000
total experiment duration (min): 39.800000
8 images will be imagined 24 times each


In [4]:
len(occluder_locations.keys())

4

In [5]:
def make_patch(occluder, img, mask):
    return Image.composite(img,occluder,mask)

def occlude_image(occluder,img,mask):
    return Image.composite(occluder,img,mask)

def random_isi(lam):
    return 1+np.random.poisson(lam=lam)  

def make_alpha_mask(size, x,y,r,fill):
    blank = make_a_blank(size, background = 0, mode='L')
    circle_mask = Draw.Draw(blank)
    circle_mask.ellipse((x-r, y-r, x+r, y+r), fill=fill)
    return blank

In [6]:
def print_frame_sequence(frame_sequence,print_to):  #+'movie_'+str(mm))
    with open(print_to,'w') as thefile:
        for item in frame_sequence:
          thefile.write("%s\n" % item)

In [7]:
def make_base_name(savedir,key,base_image_name):
    return savedir+key+'_'+base_image_name+'_base.png'

def make_patch_name(savedir,key,patch_image_name):
    return savedir+key+'_'+patch_image_name+'_patch.png'
    

####Select images

In [8]:
image_sets = {}
image_sets['northwest'] = [3167,4001]
image_sets['northeast'] = [3860,4045]
image_sets['southwest'] = [3770,3412]
image_sets['southeast'] = [3067,3277]


isi_screen = 'isi_screen.png'
imagery_screen = 'imagery_screen.png'


#####generate images

In [9]:
##occluder
occluder = make_a_blank((image_scale,image_scale),occluder_luminance,'L')

##patch masks
for key in occluder_locations.keys():
    mask =  make_alpha_mask((image_scale,image_scale),occluder_locations[key][1],occluder_locations[key][0],occluder_radius,255)
    mask.save(savedir+key+'_patch_mask.png')



##isi_screen
blank = make_a_blank(image_scale,isi_luminance,'L')
blank.save(savedir+'isi_screen.png',format='png')

##imagery_screen
blank.save(savedir+'imagery_screen.png',format='png')



In [10]:

##a loop to create the actual .pngs
for oo,os in enumerate(occluder_locations.keys()):
    images_for_current_set_and_location = image_sets[os]
    mask = Image.open(savedir+os+'_patch_mask.png')
    for bb in range(IMAGES_PER_SET):
        base_image_name = '%0.6d' %(images_for_current_set_and_location[bb])
        base_image = Image.open(picdir+base_image_name+'.png').convert('L').resize((image_scale,image_scale))
        occlude_image(occluder,base_image,mask).save(make_base_name(savedir,os,base_image_name))

        for pp in range(IMAGES_PER_SET):
            patch_image_name = '%0.6d' %(images_for_current_set_and_location[pp])
            patch_image = Image.open(picdir+patch_image_name+'.png').resize((image_scale,image_scale))
            make_patch(occluder,patch_image,mask).save(make_patch_name(savedir,os,patch_image_name))
 

####generate frame files

In [11]:

##cycle list
for run in range(RUNS):
    cycle_list = []
    for oo,os in enumerate(occluder_locations.keys()):
        images_for_current_set_and_location = image_sets[os]
        for ff in range(LOOPS):
            for bb in range(IMAGES_PER_SET):
                base_image = '%0.6d' %(images_for_current_set_and_location[bb])
                for pp in range(IMAGES_PER_SET):
                    patch_image = '%0.6d' %(images_for_current_set_and_location[pp])
                    isi_duration = random_isi(ISI_LAM)
                    base_name = make_base_name('',os,base_image)
                    patch_name = make_patch_name('',os,patch_image)
                    one_cycle = [base_name, imagery_screen, patch_name]
                    one_cycle.extend([isi_screen]*isi_duration)
                    cycle_list.append(one_cycle)
                for rr in range(MATCH_REPS_PER_SET):
                    patch_image = '%0.6d' %(images_for_current_set_and_location[bb])
                    isi_duration = random_isi(ISI_LAM)
                    base_name = make_base_name('',os,base_image)
                    patch_name = make_patch_name('',os,patch_image)
                    one_cycle = [base_name, imagery_screen, patch_name]
                    one_cycle.extend([isi_screen]*isi_duration)
                    cycle_list.append(one_cycle)
    np.random.shuffle(cycle_list)
    flat_cycle_list = [item for sublist in cycle_list for item in sublist]
    flat_cycle_list = [isi_screen]*FADE_IN_FRAMES+flat_cycle_list
    flat_cycle_list.extend([isi_screen]*FADE_OUT_FRAMES)
    print 'printing %d frames to: run_%d.txt' %(len(flat_cycle_list),run)     
    print_frame_sequence(flat_cycle_list,savedir+'run_%d.txt' %(run))

printing 209 frames to: run_0.txt
printing 207 frames to: run_1.txt
printing 210 frames to: run_2.txt
printing 212 frames to: run_3.txt
printing 208 frames to: run_4.txt
printing 208 frames to: run_5.txt
printing 211 frames to: run_6.txt
printing 209 frames to: run_7.txt
